# Repeated measure/longitudinal analysis

### Random intercept model

In [1]:
#Detection of differentially abundant taxa across different experimental groups (here is "delivery") while accounting for random effects (here is random subject effect ”studyid”)
#Example OTU table: data/ecam-table-taxa.tsv
#Example meta data: data/ecam-sample-metadata.tsv
#Identify structural zeros by specifying group_var. Here we would like to know whether there are some structural zeros across different levels of delivery

In [2]:
#install.packages('readr')
#install.packages('tidyverse')
#install.packages('compositions')

In [3]:
library(readr)
library(tidyverse)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ dplyr   1.0.7
✔ tibble  3.1.4     ✔ stringr 1.4.0
✔ tidyr   1.1.3     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [4]:
otu_data = read_tsv("/Users/dagmarschierova/MEGA/MBÚ/TNF/NGS/runs_merged/outputs/qiime_results_B/taxonomy_no_raref_Large_cohort_P/taxonomy-L7-rel-filtered1perc.tsv", skip = 1)
otu_id = otu_data$`#OTU ID`
otu_data = data.frame(otu_data[, -1], check.names = FALSE)
rownames(otu_data) = otu_id

meta_data = read_tsv("/Users/dagmarschierova/MEGA/MBÚ/TNF/NGS/runs_merged/inputs/metadata_stool_cut.tsv")[-1, ]
meta_data = meta_data %>% rename(Sample.ID = `#SampleID`)

#source("https://raw.githubusercontent.com/FrederickHuangLin/ANCOM/master/scripts/ancom_v2.1.R")
source("/Users/dagmarschierova/MEGA/MBÚ/TNF/NGS/scripts/Ancom_paired_modif.R")

Rows: 193 Columns: 280

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr   (1): #OTU ID
dbl (279): 1_BA_1, 1_BA_2, 1_BA_3, 1_BA_4, 1_BA_5, 1_BA_6, 1_BA_7, 1_BA_8, 1...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 360 Columns: 75

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (61): #SampleID, Label, Patient, Sex, BMI_category, Smoker, Disease_stat...
dbl (14): Age, BMI, Visit_num, Week_num, Week_baseline, CRP, WBC, PLT, Ferri...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Attaching package: ‘nlme’


The f

In [5]:
# Step 1: Data preprocessing

feature_table = otu_data; sample_var = "Sample.ID"; group_var = "Dg"
out_cut = 0.05; zero_cut = 0.90; lib_cut = 0; neg_lb = TRUE
prepro = feature_table_pre_process(feature_table, meta_data, sample_var, group_var, 
                                   out_cut, zero_cut, lib_cut, neg_lb)
feature_table = prepro$feature_table # Preprocessed feature table
meta_data = prepro$meta_data # Preprocessed metadata
struc_zero = prepro$structure_zeros # Structural zero info

In [6]:
Cstack_info()

size    current  direction eval_depth 
   7969177     143056          1         49

In [7]:
system("R --max-ppsize 500000", intern=TRUE)

Warning message in system("R --max-ppsize 500000", intern = TRUE):
“running command 'R --max-ppsize 500000' had status 2”


character(0)
attr(,"status")
[1] 2
attr(,"errmsg")
[1] "Resource temporarily unavailable"

In [8]:
# Step 2: ANCOM
options("expressions"=500000)

main_var = "Dg"; p_adj_method = "BH"; alpha = 0.05
adj_formula = NULL; rand_formula = "~ 1 | Patient"
control = lmeControl(maxIter = 100, msMaxIter = 100, opt = "optim")
t_start = Sys.time()
res = ANCOM(feature_table, meta_data, struc_zero, main_var, p_adj_method, 
            alpha, adj_formula, rand_formula, control = control)
t_end = Sys.time()
t_run = t_end - t_start # around 30s

write.table(res$out, file="/Users/dagmarschierova/MEGA/MBÚ/TNF/NGS/runs_merged/outputs/qiime_results_B/taxonomy_no_raref_Large_cohort_P/ANCOM2_L7-rel-Dg.tsv",
           quote=FALSE, sep='\t', row.names = FALSE)
write.table(res$dat_fig, file="/Users/dagmarschierova/MEGA/MBÚ/TNF/NGS/runs_merged/outputs/qiime_results_B/taxonomy_no_raref_Large_cohort_P/ANCOM2_L7-rel-datfig-Dg.tsv",
           quote=FALSE, sep='\t', row.names = FALSE)


ERROR: Error: protect(): protection stack overflow


In [ ]:
# Step 3: Volcano Plot

# Number of taxa except structural zeros
n_taxa = ifelse(is.null(struc_zero), nrow(feature_table), sum(apply(struc_zero, 1, sum) == 0))
# Cutoff values for declaring differentially abundant taxa
cut_off = c(0.9 * (n_taxa -1), 0.8 * (n_taxa -1), 0.7 * (n_taxa -1), 0.6 * (n_taxa -1))
names(cut_off) = c("detected_0.9", "detected_0.8", "detected_0.7", "detected_0.6")

# Annotation data
dat_ann = data.frame(x = min(res$fig$data$x), y = cut_off["detected_0.7"], label = "W[0.7]")

fig = res$fig + 
  geom_hline(yintercept = cut_off["detected_0.7"], linetype = "dashed") + 
  geom_text(data = dat_ann, aes(x = x, y = y, label = label), 
            size = 4, vjust = -0.5, hjust = 0, color = "orange", parse = TRUE)
fig   


In [ ]:
#png(filename="/Users/dagmarschierova/MEGA/MBÚ/TNF/NGS/runs_merged/outputs/qiime_results_B/taxonomy_no_raref_Large_cohort/ANCOM2_L7-rel-Disease_status-pic.png",
#   res=300, width=1500, height=1500,)
#plot(res$fig)
#dev.off()